# Package

In [1]:
#Torch related package
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable

In [2]:
#cuda related package
import torch.cuda
import torch.backends.cudnn as cudnn
print(torch.backends.cudnn.version())

None


In [2]:
# Other package
import time

In [6]:
#%run Annexe.ipynb #Good trick to launch another notebook

# Commentaire pour la suite (TODO)

Le batch normalisation, il y en existe plusieurs, regarder exactement lequel on veut.

On peut enlever le biais dans les convolutions avant la normalisation

Le diminution de la taille des images, pas clair cette division par deux....

# Entrainement 

In [ ]:
class gridNet(nn.Module):
    
    """
    (1) = nInput : number of features maps for the input
    (2) = nOutput : number of features maps for the output
    (3) = nColumns : number of columns of the gridNet, this number should be divisible by two.
    It count the number of bloc +1
    (4) = nFeatMaps : number of feature at each row of the gridNet
    (5) = dropFactor : factor witch control the dropout of an entire bloc 
    """
    def __init__(self, input_dim, hidden_dim, steps=1):
        super(testNet, self).__init__()
        self.steps = steps
        self.i2h = ListModle(self, 'i2h_')
        self.h2h = ListModle(self, 'h2h_')
        for i in range(steps):
            self.i2h.append(nn.Linear(input_dim, hidden_dim))
            self.h2h.append(nn.Linear(hidden_dim, hidden_dim))

    def forward(self, inputs, hidden):
        for inp, i2h, h2h in zip(inputs, self.i2h, self.h2h):
            print(4)
            hidden = F.tanh(i2h(inp) + h2h(hidden))
        return hidden
    
    
    def __init__(self,nInputs, nOutputs, nColumns, nFeatMaps, dropFactor):
        super(gridNet, self).__init__()
        len_nfeatureMaps = len(nFeatMaps)
        self.nColumns = nColumns
        self.nFeatMaps = nFeatMaps
        
        
        # A normalisation before any computation
        self.batchNormInitial = nn.BatchNorm2d(num_features = nInputs, eps=1e-05, momentum=0.1,affine=True)

        # The first convolution before entering into the grid.
        self.firstConv = firstConv(nInputs = nInputs, nOutputs = nFeatMaps[0])
        
        
        # We create the Grid 
        for i in range(len(nFeatMaps)):
            for j in range(nColumns):
                #We don t creat a residual bloc on the last column
                if(j < (nColumns - 1)):
                    setattr(self, "convSequence" + str(i) + "_" + str(j) + "to" + str(i) + "_" + str(j + 1),
                            convSequence(nFeatMaps[i], nFeatMaps[i],dropFactor))
                
                #We creat subSampling only on half of the grid and not in the last row
                if(j < (nColumns // 2) and i < (len(nFeatMaps)-1)):
                    setattr(self, "subSamplingSequence" + str(i) + "_" + str(j) + "to" + str(i + 1) + "_" + str(j),
                            subSamplingSequence(nFeatMaps[i], nFeatMaps[i+1]))
                
                if(j >= (nColumns // 2) and i > 0):
                    setattr(self, "upSamplingSequence" + str(i) + "_" + str(j) + "to" + str(i - 1) + "_" + str(j),
                            upSamplingSequence(nFeatMaps[i], nFeatMaps[i-1]))

        # The last convolution before the result.
        self.lastConv = lastConv(nInputs = nFeatMaps[0], nOutputs = nOutputs)
        
    def forward(self, x):
        
        len_nfeatureMaps = len(self.nFeatMaps)

        x = self.batchNormInitial(x)
        x = self.firstConv(x)
        
        X = [[0 for i in range(self.nColumns)] for j in range(len_nfeatureMaps)] 
        X[0][0] = x
        

        for i in range(1,len_nfeatureMaps-1):
            X[i][0] = getattr(self,"subSamplingSequence" + str(i-1) + "_" + str(0) + "to" + str(i) + "_" + str(0))

        for j in range(1,nColumns//2):
            for i in range(len_nfeatureMaps):
                    X[i][j] = self.X_residual[i-1][j]
        
        for j in range(self.nColumns):
            X[0][j] = getattr(self, prefix + str(i))
            For example, getattr(x, 'foobar') is equivalent to x.foobar.
    
    
        return X[0][self.nColumns-1]

    
torch.manual_seed(7)
    
network = gridNet(nInputs = 3,nOutputs = 3, nColumns = 5, nFeatMaps = [3,6,12,24], dropFactor = 0.1)
#print(network)
a = torch.randn(2, 3, 33, 33)
inputs = Variable(a)
targets = Variable(a)


outputs = network(inputs)
print(outputs.size())


criterion = nn.MSELoss()
loss = criterion(outputs, targets)
print(loss)

# create your optimizer
optimizer = optim.SGD(network.parameters(), lr=0.01)

# in your training loop:
optimizer.zero_grad()   # zero the gradient buffers
for i in range(100):
    outputs = network(inputs)
    loss = criterion(outputs, target)
    loss.backward()
    optimizer.step()    # Does the update
    #if(i%100==99):
        #print(loss.data[0])
    

In [226]:
class MatriceModuleResidual(object):
    #Should work with all kind of module
    def __init__(self, module, prefix, *args):
        self.module = module
        self.prefix = prefix
        for new_module in args:
            self.append(new_module)

    def append(self, new_module,i1,j1,i2,j2):
        if not isinstance(new_module, nn.Module):
            raise ValueError('Not a Module')
        else:
            self.module.add_module(self.prefix + str(i) + str(j) + str(i+1) + str(j+1), new_module)

    def __getitem__(self, i,j):
        return getattr(self.module, self.prefix + str(i) + str(j) + str(i+1) + str(j+1))

In [221]:
class ListModle(object):
    #Should work with all kind of module
    def __init__(self, module, prefix, *args):
        self.module = module
        self.prefix = prefix
        self.num_module = 0
        for new_module in args:
            self.append(new_module)

    def append(self, new_module):
        if not isinstance(new_module, nn.Module):
            raise ValueError('Not a Module')
        else:
            self.module.add_module(self.prefix + str(self.num_module), new_module)
            self.num_module += 1

    def __len__(self):
        return self.num_module

    def __getitem__(self, i):
        if i < 0 or i >= self.num_module:
            raise IndexError('Out of bound')
        return getattr(self.module, self.prefix + str(i))


class testNet(nn.Module):
    def __init__(self, input_dim, hidden_dim, steps=1):
        super(testNet, self).__init__()
        self.steps = steps
        self.i2h = ListModle(self, 'i2h_')
        self.h2h = ListModle(self, 'h2h_')
        for i in range(steps):
            self.i2h.append(nn.Linear(input_dim, hidden_dim))
            self.h2h.append(nn.Linear(hidden_dim, hidden_dim))

    def forward(self, inputs, hidden):
        for inp, i2h, h2h in zip(inputs, self.i2h, self.h2h):
            print(4)
            hidden = F.tanh(i2h(inp) + h2h(hidden))
        return hidden

net = testNet(128, 256, 3)
print(net)
inp = Variable(torch.randn(6, 4, 128))
init = Variable(torch.randn(4, 256))
out = net(inp, init)
print(out.size())

testNet (
  (i2h_0): Linear (128 -> 256)
  (h2h_0): Linear (256 -> 256)
  (i2h_1): Linear (128 -> 256)
  (h2h_1): Linear (256 -> 256)
  (i2h_2): Linear (128 -> 256)
  (h2h_2): Linear (256 -> 256)
)
4
4
4
torch.Size([4, 256])


In [214]:
class AttrProxy(object):
    """Translates index lookups into attribute lookups."""
    def __init__(self, module, prefix):
        self.module = module
        self.prefix = prefix

    #def __getitem__(self, i):
    #    return getattr(self.module, self.prefix + str(i))
    def __getitem__(self, idx):
        if idx < 0 or idx >= len(self._modules):
            raise IndexError('index {} is out of range'.format(idx))
        it = iter(self._modules.values())
        for i in range(idx):
            next(it)
        return next(it)

class testNet(nn.Module):
    def __init__(self, input_dim, hidden_dim, steps=1):
        super(testNet, self).__init__()
        self.steps = steps
        for i in range(steps):
            self.add_module('i2h_' + str(i), nn.Linear(input_dim, hidden_dim))
            self.add_module('h2h_' + str(i), nn.Linear(hidden_dim, input_dim))
        self.i2h = AttrProxy(self, 'i2h_')
        self.h2h = AttrProxy(self, 'h2h_')

    def forward(self, inputs):
        # here, use self.i2h[t] and self.h2h[t] to index 
        # input2hidden and hidden2hidden modules for each step,
        # or loop over them, like in the example below
        # (assuming first dim of input is sequence length)
        x=0
        y=0
        print(self.i2h)
        for inp, i2h, h2h in zip(inputs, self.i2h, self.h2h):
            x += F.tanh(i2h(inputs))
        i = 0
        for i2ha in self.i2h:
            print(self.i2h)
            print(i)
            i = i + 1
            print(i2ha)
            y += F.tanh(i2ha(inputs))
            
        #for i2h,h2h in zip(self.i2h,self.h2h):
            #print(i2h)
            #y = F.tanh(i2h(inputs))
            
        return y
print(testNet(3,3,5))
z = testNet(3,3,5)
a = torch.randn(3, 3)
inputs = Variable(a)
out = z(inputs)
print(out.size())

testNet (
  (i2h_0): Linear (3 -> 3)
  (h2h_0): Linear (3 -> 3)
  (i2h_1): Linear (3 -> 3)
  (h2h_1): Linear (3 -> 3)
  (i2h_2): Linear (3 -> 3)
  (h2h_2): Linear (3 -> 3)
  (i2h_3): Linear (3 -> 3)
  (h2h_3): Linear (3 -> 3)
  (i2h_4): Linear (3 -> 3)
  (h2h_4): Linear (3 -> 3)
)


AttributeError: 'AttrProxy' object has no attribute '_modules'

In [37]:
class firstConv(nn.Module):
    
    """
    (1) = nInput : number of features map for the input
    (2) = nOutput : number of features map for the output
    (3) = nMiddleput : number of features map after the first Convolution
    """
    def __init__(self,nInputs,nOutputs):
        super(firstConv, self).__init__()

        self.conv1 = nn.Conv2d(in_channels = nInputs, out_channels = nOutputs,
                            kernel_size = (3,3), stride=(1,1), padding=(1,1), dilation=1, groups=1, bias=True)
        
        self.batch1 = nn.BatchNorm2d(num_features = nOutputs, eps=1e-05, momentum=0.1,affine=True)
        
        self.conv2 = nn.Conv2d(in_channels = nOutputs, out_channels = nOutputs,
                            kernel_size = (3,3), stride=(1,1), padding=(1,1), dilation=1, groups=1, bias=True)
        
        self.batch2 = nn.BatchNorm2d(num_features = nOutputs, eps=1e-05, momentum=0.1,affine=True)
                
        
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.batch1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = self.batch2(x)
        x = F.relu(x)
        return x

    
net = firstConv(nInputs = 3,nOutputs = 3)
print(net)
a = torch.randn(2, 3, 16, 16)
inputs = Variable(a)
out = net(inputs)
print(out)

target = Variable(a)  # a dummy target, for example
criterion = nn.MSELoss()

loss = criterion(out, target)
print(loss)


firstConv (
  (conv1): Conv2d(3, 3, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (batch1): BatchNorm2d(3, eps=1e-05, momentum=0.1, affine=True)
  (conv2): Conv2d(3, 3, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (batch2): BatchNorm2d(3, eps=1e-05, momentum=0.1, affine=True)
  (dropOut): Dropout2d (p=0.5)
)
Variable containing:
(0 ,0 ,.,.) = 
  0.0000  0.0000  0.0000  ...   0.2448  0.0000  0.0000
  0.2194  0.0000  0.0000  ...   0.4013  0.1989  0.1344
  0.1668  0.3783  0.0000  ...   0.0000  0.1836  0.0000
           ...             ⋱             ...          
  0.2413  0.4212  0.0000  ...   0.3038  0.0000  0.0635
  0.0000  0.0000  0.0000  ...   0.3256  0.0000  0.0000
  0.0000  0.1842  0.3408  ...   0.3764  0.1104  0.0000

(0 ,1 ,.,.) = 
  0.0000  0.1016  0.0749  ...   0.0000  0.0181  0.0000
  0.0494  0.0005  0.0644  ...   0.1323  0.0904  0.1407
  0.0000  0.0000  0.0000  ...   0.0499  0.0000  0.0421
           ...             ⋱             ...          
  0.0000  0.1420  

In [128]:
if torch.cuda.device_count() > 1:
  print("Let's use", torch.cuda.device_count(), "GPUs!")

In [185]:
class double_conv(nn.Module):
    '''(conv => BN => ReLU) * 2'''
    def __init__(self, in_ch, out_ch):
        super(double_conv, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_ch, out_ch, 3, padding=1),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_ch, out_ch, 3, padding=1),
            nn.BatchNorm2d(out_ch),
        )

    def forward(self, x):
        x = self.conv(x)
        x = f.ReLU(inplace=True)
        return x
print(double_conv(4,5))

double_conv (
  (conv): Sequential (
    (0): Conv2d(4, 5, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(5, eps=1e-05, momentum=0.1, affine=True)
    (2): ReLU (inplace)
    (3): Conv2d(5, 5, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(5, eps=1e-05, momentum=0.1, affine=True)
  )
)


In [245]:
class firstConv(nn.Module):
    
    """
    (1) = nInput : number of features map for the input
    (2) = nOutput : number of features map for the output
    (3) = nMiddleput : number of features map after the first Convolution
    """
    def __init__(self,nInputs,nOutputs):
        super(firstConv, self).__init__()

        self.conv1 = nn.Conv2d(in_channels = nInputs, out_channels = nOutputs,
                            kernel_size = (3,3), stride=(1,1), padding=(1,1), dilation=1, groups=1, bias=True)
        
        self.batch1 = nn.BatchNorm2d(num_features = nOutputs, eps=1e-05, momentum=0.1,affine=True)
        
        self.conv2 = nn.Conv2d(in_channels = nOutputs, out_channels = nOutputs,
                            kernel_size = (3,3), stride=(1,1), padding=(1,1), dilation=1, groups=1, bias=True)
        
        self.batch2 = nn.BatchNorm2d(num_features = nOutputs, eps=1e-05, momentum=0.1,affine=True)
        
        self.dropOut = nn.Dropout2d(0.5)
        
        
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.batch1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = self.batch2(x)
        x = F.relu(x)
        return x

    
net = firstConv(nInputs = 3,nOutputs = 3)
print(net)


firstConv (
  (conv1): Conv2d(3, 3, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (batch1): BatchNorm2d(3, eps=1e-05, momentum=0.1, affine=True)
  (conv2): Conv2d(3, 3, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (batch2): BatchNorm2d(3, eps=1e-05, momentum=0.1, affine=True)
  (dropOut): Dropout2d (p=0.5)
)


# CityscapesLoader

#  GridNet

In [2]:
class color():
    convolution        = "darkgoldenrod1"
    subSampling        = "darkgoldenrod" 
    fullConvolution    = "firebrick1"
    upSampling         = "firebrick"
    batchNormalization = "deepskyblue3"
    relu               = "darkolivegreen3"
    add                = "bisque3"
    dropout            = "darkviolet"

In [3]:
class firstConv(nn.Module):
    
    """
    (1) = nInputs : number of features map for the input
    (2) = nOutputs : number of features map for the output
    """
    def __init__(self,nInputs,nOutputs):
        super(firstConv, self).__init__()

        self.conv1 = nn.Conv2d(in_channels = nInputs, out_channels = nOutputs,
                            kernel_size = (3,3), stride=(1,1), padding=(1,1), dilation=1, groups=1, bias=True)
        
        self.batch1 = nn.BatchNorm2d(num_features = nOutputs, eps=1e-05, momentum=0.1,affine=True)
        
        self.ReLU1 = nn.ReLU()
        
        self.conv2 = nn.Conv2d(in_channels = nOutputs, out_channels = nOutputs,
                            kernel_size = (3,3), stride=(1,1), padding=(1,1), dilation=1, groups=1, bias=True)
        
        self.batch2 = nn.BatchNorm2d(num_features = nOutputs, eps=1e-05, momentum=0.1,affine=True)
                
        self.ReLU2 = nn.ReLU()
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.batch1(x)
        x = self.ReLU1(x)
        x = self.conv2(x)
        x = self.batch2(x)
        x = self.ReLU2(x)
        return x

    
net = firstConv(nInputs = 3,nOutputs = 3)
print(net)


firstConv (
  (conv1): Conv2d(3, 3, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (batch1): BatchNorm2d(3, eps=1e-05, momentum=0.1, affine=True)
  (ReLU1): ReLU ()
  (conv2): Conv2d(3, 3, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (batch2): BatchNorm2d(3, eps=1e-05, momentum=0.1, affine=True)
  (ReLU2): ReLU ()
)


In [133]:
class convSequence(nn.Module):
    
    """
    (1) = nInput : number of features map for the input
    (2) = nOutput : number of features map for the output
    (3) = dropFactor : number of features map after the first Convolution
    """
    def __init__(self,nInputs,nOutputs,dropFactor):
        super(convSequence, self).__init__()
        self.batch1 = nn.BatchNorm2d(num_features = nInputs, eps=1e-05, momentum=0.1,affine=True)

        self.conv1 = nn.Conv2d(in_channels = nInputs, out_channels = nOutputs,
                            kernel_size = (3,3), stride=(1,1), padding=(1,1), dilation=1, groups=1, bias=True)
        
        self.ReLU1 = nn.ReLU()
        
        self.batch2 = nn.BatchNorm2d(num_features = nOutputs, eps=1e-05, momentum=0.1,affine=True)
        
        self.conv2 = nn.Conv2d(in_channels = nOutputs, out_channels = nOutputs,
                            kernel_size = (3,3), stride=(1,1), padding=(1,1), dilation=1, groups=1, bias=True)
        
        self.ReLU2 = nn.ReLU()
        
        
    def forward(self, x_init):
        x = self.batch1(x_init)
        x = self.conv1(x)
        x = self.ReLU1(x)
        x = self.batch2(x)
        x = self.conv2(x)
        x = self.ReLU2(x)
        x = (torch.rand(1) > dropFactor)*x
        x = x_init + x
        return x

    
net = convSequence(nInputs = 3,nOutputs = 3,dropFactor = 0.1)
print(net)

convSequence (
  (batch1): BatchNorm2d(3, eps=1e-05, momentum=0.1, affine=True)
  (conv1): Conv2d(3, 3, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (ReLU1): ReLU ()
  (batch2): BatchNorm2d(3, eps=1e-05, momentum=0.1, affine=True)
  (conv2): Conv2d(3, 3, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (ReLU2): ReLU ()
)


In [102]:
class subSamplingSequence(nn.Module):
    
    """
    (1) = nInput : number of features map for the input
    (2) = nOutput : number of features map for the output
    This class represente a bloc that reduce the resolution of each feature map(factor2)
    """
    def __init__(self, nInputs, nOutputs):
        super(subSamplingSequence, self).__init__()
        
        self.batch1 = nn.BatchNorm2d(num_features = nInputs, eps=1e-05, momentum=0.1,affine=True)

        self.conv1 = nn.Conv2d(in_channels = nInputs, out_channels = nOutputs,
                            kernel_size = (3,3), stride=(2,2), padding=(1,1), dilation=1, groups=1, bias=True)
        
        self.ReLU1 = nn.ReLU()
        
        self.batch2 = nn.BatchNorm2d(num_features = nOutputs, eps=1e-05, momentum=0.1,affine=True)
        
        self.conv2 = nn.Conv2d(in_channels = nOutputs, out_channels = nOutputs,
                            kernel_size = (3,3), stride=(1,1), padding=(1,1), dilation=1, groups=1, bias=True)

        self.ReLU2 = nn.ReLU()

    def forward(self, x):
        x = self.batch1(x)
        x = self.conv1(x)
        x = self.ReLU1(x)
        x = self.batch2(x)
        x = self.conv2(x)
        x = self.ReLU2(x)
        return x

    
network = subSamplingSequence(nInputs = 3,nOutputs = 6)
print(network)
a = torch.randn(2, 3, 7, 7)
inputs = Variable(a)
out = network(inputs)
print(out.size())

subSamplingSequence (
  (batch1): BatchNorm2d(3, eps=1e-05, momentum=0.1, affine=True)
  (conv1): Conv2d(3, 6, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (ReLU1): ReLU ()
  (batch2): BatchNorm2d(6, eps=1e-05, momentum=0.1, affine=True)
  (conv2): Conv2d(6, 6, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (ReLU2): ReLU ()
)
torch.Size([2, 6, 4, 4])


In [104]:
class upSamplingSequence(nn.Module):
    
    """
    (1) = nInput : number of features map for the input
    (2) = nOutput : number of features map for the output
    This class represente a bloc that increase the resolution of each feature map(factor2)
    """
    def __init__(self, nInputs, nOutputs):
        super(upSamplingSequence, self).__init__()
        
        self.batch1 = nn.BatchNorm2d(num_features = nInputs, eps=1e-05, momentum=0.1,affine=True)

        self.convTranspose1 = nn.ConvTranspose2d(in_channels = nInputs, out_channels = nOutputs,
                            kernel_size = (3,3), stride=(2,2), padding=(1,1), dilation=1, groups=1, bias=True)
        
        self.ReLU1 = nn.ReLU()
        
        self.batch2 = nn.BatchNorm2d(num_features = nOutputs, eps=1e-05, momentum=0.1,affine=True)
        
        self.conv2 = nn.Conv2d(in_channels = nOutputs, out_channels = nOutputs,
                            kernel_size = (3,3), stride=(1,1), padding=(1,1), dilation=1, groups=1, bias=True)

        self.ReLU2 = nn.ReLU()

        
    def forward(self, x):
        x = self.batch1(x)
        x = self.convTranspose1(x)
        x = self.ReLU1(x)
        x = self.batch2(x)
        x = self.conv2(x)
        x = self.ReLU2(x)
        return x


network = upSamplingSequence(nInputs = 6,nOutputs = 3)
print(network)
a = torch.randn(2, 6, 16, 16)
inputs = Variable(a)
out = network(inputs)
print(out.size())

upSamplingSequence (
  (batch1): BatchNorm2d(6, eps=1e-05, momentum=0.1, affine=True)
  (convTranspose1): ConvTranspose2d(6, 3, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (ReLU1): ReLU ()
  (batch2): BatchNorm2d(3, eps=1e-05, momentum=0.1, affine=True)
  (conv2): Conv2d(3, 3, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (ReLU2): ReLU ()
)
torch.Size([2, 3, 31, 31])


In [7]:
class lastConv(nn.Module):
    
    """
    (1) = nInputs : number of features map for the input
    (2) = nOutputs : number of features map for the output
    This class represente the last Convolution of the network before the prediction
    """
    def __init__(self,nInputs,nOutputs):
        super(lastConv, self).__init__()

        self.conv1 = nn.Conv2d(in_channels = nInputs, out_channels = nOutputs,
                            kernel_size = (3,3), stride=(1,1), padding=(1,1), dilation=1, groups=1, bias=True)
        
        self.batch1 = nn.BatchNorm2d(num_features = nOutputs, eps=1e-05, momentum=0.1,affine=True)
        
        self.ReLU1 = nn.ReLU()
        
        self.conv2 = nn.Conv2d(in_channels = nOutputs, out_channels = nOutputs,
                            kernel_size = (3,3), stride=(1,1), padding=(1,1), dilation=1, groups=1, bias=True)
        
        self.batch2 = nn.BatchNorm2d(num_features = nOutputs, eps=1e-05, momentum=0.1,affine=True)
                
        self.ReLU2 = nn.ReLU()

        
    def forward(self, x):
        x = self.conv1(x)
        x = self.batch1(x)
        x = self.ReLU1(x)
        x = self.conv2(x)
        x = self.batch2(x)
        x = self.ReLU2(x)
        return x

    
net = lastConv(nInputs = 3,nOutputs = 3)
print(net)


lastConv (
  (conv1): Conv2d(3, 3, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (batch1): BatchNorm2d(3, eps=1e-05, momentum=0.1, affine=True)
  (ReLU1): ReLU ()
  (conv2): Conv2d(3, 3, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (batch2): BatchNorm2d(3, eps=1e-05, momentum=0.1, affine=True)
  (ReLU2): ReLU ()
)


In [175]:
class gridNet(nn.Module):
    
    """
    (1) = nInput : number of features maps for the input
    (2) = nOutput : number of features maps for the output
    (3) = nColumns : number of columns of the gridNet, this number should be divisible by two.
    It count the number of bloc +1
    (4) = nFeatMaps : number of feature at each row of the gridNet
    (5) = dropFactor : factor witch control the dropout of an entire bloc 
    """
    def __init__(self,nInputs, nOutputs, nColumns, nFeatMaps, dropFactor):
        super(gridNet, self).__init__()
        len_nfeatureMaps = len(nFeatMaps)
        self.nColumns = nColumns
        self.nFeatMaps = nFeatMaps
        
        
        # A normalisation before any computation
        self.batchNormInitial = nn.BatchNorm2d(num_features = nInputs, eps=1e-05, momentum=0.1,affine=True)

        # The first convolution before entering into the grid.
        self.firstConv = firstConv(nInputs = nInputs, nOutputs = nFeatMaps[0])
        
        
        # We create the Grid. We will creat conv and sub/up sequences with different name.
        # The name is : "sequenceName" + starting position of the sequence(i,j) + "to" + ending position (k,l)
        for i in range(len(nFeatMaps)):
            for j in range(nColumns):
                #We don t creat a residual bloc on the last column
                if(j < (nColumns - 1)):
                    setattr(self, "convSequence" + str(i) + "_" + str(j) + "to" + str(i) + "_" + str(j + 1),
                            convSequence(nFeatMaps[i], nFeatMaps[i],dropFactor))
                
                #We creat subSampling only on half of the grid and not in the last row
                if(j < (nColumns // 2) and i < (len(nFeatMaps)-1)):
                    setattr(self, "subSamplingSequence" + str(i) + "_" + str(j) + "to" + str(i + 1) + "_" + str(j),
                            subSamplingSequence(nFeatMaps[i], nFeatMaps[i+1]))
                
                #Welook a the other half but not the first row
                if(j >= (nColumns // 2) and i > 0):
                    setattr(self, "upSamplingSequence" + str(i) + "_" + str(j) + "to" + str(i - 1) + "_" + str(j),
                            upSamplingSequence(nFeatMaps[i], nFeatMaps[i-1]))

        # The last convolution before the result.
        self.lastConv = lastConv(nInputs = nFeatMaps[0], nOutputs = nOutputs)
        
    def forward(self, x):
        len_nfeatureMaps = len(self.nFeatMaps)

        # A normalisation before any computation
        x = self.batchNormInitial(x)
        # The first convolution before entering into the grid.
        x = self.firstConv(x)
        
        # X is the matrix that represente the values of the features maps at the point (i,j) in the grid.
        X = [[0 for i in range(self.nColumns)] for j in range(len_nfeatureMaps)]
        #The input of the grid is on (0,0)
        X[0][0] = x
        
        #We did the special case for j=0
        for i in range(1,len_nfeatureMaps-1):
            X[i][0] = getattr(self,"subSamplingSequence" + str(i-1) + "_" + str(0) + "to" + str(i) + "_" + str(0))

        for j in range(1,nColumns//2):
            for i in range(len_nfeatureMaps):
                    X[i][j] = self.X_residual[i-1][j]

        for j in range(self.nColumns):
            X[0][j] = getattr(self.prefix + str(i))
    
    
        return X[0][self.nColumns-1]

    
torch.manual_seed(7)
    
network = gridNet(nInputs = 3,nOutputs = 3, nColumns = 5, nFeatMaps = [3,6,12,24], dropFactor = 0.1)
#print(network)
a = torch.randn(2, 3, 33, 33)
inputs = Variable(a)
targets = Variable(a)


outputs = network(inputs)
print(outputs.size())


criterion = nn.MSELoss()
loss = criterion(outputs, targets)
print(loss)

# create your optimizer
optimizer = optim.SGD(network.parameters(), lr=0.01)

# in your training loop:
optimizer.zero_grad()   # zero the gradient buffers
for i in range(100):
    outputs = network(inputs)
    loss = criterion(outputs, target)
    loss.backward()
    optimizer.step()    # Does the update
    #if(i%100==99):
        #print(loss.data[0])
    

TypeError: 'int' object is not callable

# MiniBatch

# Trainer

# ZeroTarget